**Starbucks Dataset**

[Source](https://github.com/skymoonlight/biased-sents-annotation)


Labels from 1 to 4 from 4-5 annotators for 46 articles on 842 sentences.
'neutral and not biased, slightly biased, biased, and very biased'
Averaged the annotations from different annotators.



```
LABEL (binary):
NEUTRAL - 0
BIASED - 1
```

Everything under a rating of `1.5` is neutral; everything above is biased

```
CATEGORY (average value NOT categorical):
NEUTRAL - 0
SLIGHTLY BIASED - 1
BIASED - 2
VERY BIASED - 3
```

In [1]:
import os
import sys
import pandas as pd
from prep_collection import PrepCollection as prep

In [2]:
wdr_path = os.path.dirname(os.path.dirname(os.getcwd()))
ds_raw_path = os.path.join(wdr_path + "/Datasets/Linguistic Bias/Starbucks/Sora_LREC2020_biasedsentences.csv")

In [3]:
def preprocess_starbucks(wdr_path, ds_raw_path):
    df_original = pd.read_csv(ds_raw_path)
    df_text = df_original.groupby('id_article').min().reset_index()
    df_average_label = df_original.groupby('id_article').mean()
    ds = []
    article_id = 0
    for row in range(len(df_text)):
        sentence_id = 0
        for sent_index in range(20):
            category = df_average_label.loc[row+1, str(sent_index)]
            if category < 1.5:
                label = 0
            elif category >= 1.5:
                label = 1
            else:
                raise ValueError
            text = prep.prepare_text(str(df_text.loc[row, 's' + str(sent_index)])[4:])
            id = str(article_id) + '-' + str(sentence_id)
            if text != "":  # Some articles are shorter than 19 sentences
                sub_lst = [id, text,  label, category-1] # original scale goes from 1 to 4
                ds.append(sub_lst)
            sentence_id += 1
        article_id += 1
    df = pd.DataFrame(ds, columns=['id', 'text', 'label', 'category'])
    df.to_csv(os.path.join(wdr_path + "/Preprocessed_Datasets/038-Starbucks.csv"))

In [4]:
preprocess_starbucks(wdr_path, ds_raw_path)